In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [3]:
# Read the text file
with open('/content/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    data = file.read()

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

In [7]:
total_words

8200

This is the total number of unique words in data

In [9]:
input_sequences = []
for line in data.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [10]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [11]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [12]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [13]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 215s 71ms/step - loss: 6.2391 - accuracy: 0.0768
Epoch 2/100
3010/3010 [==============================] - 212s 71ms/step - loss: 5.5118 - accuracy: 0.1246
Epoch 3/100
3010/3010 [==============================] - 216s 72ms/step - loss: 5.1238 - accuracy: 0.1478
Epoch 4/100
3010/3010 [==============================] - 214s 71ms/step - loss: 4.7972 - accuracy: 0.1652
Epoch 5/100
3010/3010 [==============================] - 211s 70ms/step - loss: 4.4970 - accuracy: 0.1849
Epoch 6/100
3010/3010 [==============================] - 211s 70ms/step - loss: 4.2112 - accuracy: 0.2048
Epoch 7/100
3010/3010 [==============================] - 212s 70ms/step - loss: 3.9400 - accuracy: 0.2292
Epoch 8/100
3010/3010 [==============================] - 212s 70ms/step - loss: 3.6830 - accuracy: 0.2590
Epoch 9/100
3010/3010 [==============================] - 211s 70ms/step - loss: 3.4416 - accuracy: 0.2894
Epoch 10/100
3010/3010 [======================

In [24]:
seed_text = "Everynight in my dreams I am"
next_words = 7

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 25ms/step
Everynight in my dreams I am able to give you my uncle's perplexity
